# CI Portfolio Project 5 - Filter Maintenance Predictor 2022
## **Data Cleaning Notebook**

## Objectives

*   Confirm / Evaluate missing data
*   Clean data in preparation for analysis

### Inputs

1. Test Dataset : `outputs/datasets/collection/PredictiveMaintenanceTest.csv`

2. Train Dataset : `outputs/datasets/collection/PredictiveMaintenanceTrain.csv`

### Outputs

* Generate cleaned Train and Test sets, both saved under `outputs/datasets/cleaned`

### Conclusions

  * Data Cleaning Pipeline
  * Drop Variables as Required
  <!-- `['customerID', 'TotalCharges' ]` -->

---

# Change working directory

In [ ]:
import os
current_dir = os.getcwd()
current_dir

In [ ]:
os.chdir(os.path.dirname(current_dir))
print("Current directory set to new location")

In [ ]:
current_dir = os.getcwd()
current_dir

---

# Load Collection Data

In [ ]:
import pandas as pd
df_train = pd.read_csv(f'outputs/datasets/collection/PredictiveMaintenanceTrain.csv')
df_test = pd.read_csv(f'outputs/datasets/collection/PredictiveMaintenanceTest.csv')

In [ ]:
df_train.info()

In [ ]:
df_test.info()

---

# Data Exploration

### Check for Missing Data

To confirm we don't have variables with missing data, and if we do; discover their distribution and shape.
* Note: we are aware that the **df_train** dataset does not have values for `RUL`, so both sets are checked separately

If we tried to combine the sets to check, it would indicate `RUL` has missing values like so: 

In [ ]:
df_total = pd.concat([df_train, df_test])
vars_with_missing_data = df_total.columns[df_total.isna().sum() > 0].to_list()
vars_with_missing_data

#### To check both datasets for missing data at the same time

Define a handy function to identify which dataframe

In [ ]:
def name_dataframe(data):
    """ To identify which dataframe is being accessed """
    name =[n for n in globals() if globals()[n] is data][0]
    print('Dataframe name: %s' % name)

Check for missing data & return error information if there is

In [ ]:
from pandas_profiling import ProfileReport

for df in (df_train, df_test):
    vars_with_missing_data = df.columns[df.isna().sum() > 0].to_list()
    if vars_with_missing_data:
        profile = ProfileReport(df=df[vars_with_missing_data], minimal=True)
        profile.to_notebook_iframe()
    else:
        name_dataframe(df)
        print('There are no variables with missing data')

---

## Outliers of change in differential pressure measures `change_DP`

In each bin we notice that the change_DP measure, the size and direction of first few observations indicated they may be outliers. We have considered three main methods to deal with outliers:
* Dropping the outliers.
* Winsorize method.
* Log transformation.

In [ ]:
# %matplotlib inline
# df_train['change_DP'].plot(kind="boxplot", title='Proportion of Dust Classes in df_train\n', xlabel='\nObservations', ylabel='Dust Class')


import seaborn as sns
import matplotlib.pyplot as plt

sns.boxplot(x = df_train['change_DP'])
plt.show()

---

## Evenly distribute dataset by `Dust` type

Both the train and test sets supplied have data distributed unevenly between 50 test bins. To account for this we wish to assess the measures of central tendency for each Dust class, with the aim of reducing the data size to a more evenly proportioned one between classes.

#### **Train** Dataset

**Considerations**

* The proportion of data that **has reached filter failure** is represented by how close `filter_balance` is to zero or less. 
    * Data with filter_balance values approaching zero may be worth keeping and will make part of our heuristic decision process.
* Notwithstanding that the **mean** is the most frequently used measure of central tendency because it uses all values in the data set to give you an average
    * For data from skewed distributions (like `differential_pressure`), the **median** is better than the mean because it isn’t influenced by extremely large values.

In the following calculation, we see a summary of the top ten `Data_No` bins where `differential_pressure` observations that have made it to the **600 Pa** (the point of filter failure).

In [ ]:
last_row_train = df_train[df_train.Data_No != df_train.Data_No.shift(-1)]
# last_row_descending = last_row_train.sort_values(by='Dust', ascending=True)
last_row_descending = last_row_train.sort_values(by='Differential_pressure', ascending=False)
last_row_descending.head(n=10)

Note the diagram below showing proportions of `Dust` variable in the **df_train** dataset.
* It shows a disproportionate mix between classes. This will be the first dataset we tidy up.

In [ ]:
%matplotlib inline

category_totals = df_train.groupby('Dust')['Differential_pressure'].count().sort_values()
category_totals.plot(kind="barh", title='Proportion of Dust Classes in df_train\n', xlabel='\nObservations', ylabel='Dust Class')
category_totals

---

## Represent Central Tendency

#### Our next aim is to 
* Fill these bins with data that best represents a central tendency.
* Make the size of each bin around **9400** observations (similar to the A4 Coarse Dust class bin) 

#### Procedure
* Include a comparison to how far each `differential_pressure` measure **deviates** or how far it is from the **.median()** value of the bin.
* Ordered by `filter_balance` showing sets with data closest to 600 Pa `differential_pressure`.
* Include comparison to median
* Add a cumulative measure of Data_No's to use as a ranking
* Create a dataframe of the A3 Medium Dust : **1.025**

Add a calculation of Standard Deviation to **df_train** test set

In [ ]:
std_group = df_train.groupby('Data_No').std()
std_group.index.name = None
std_group['Data_No'] = std_group.index
map_std = df_train['Data_No'].map(std_group.set_index('Data_No')['Differential_pressure'])
df_train['std_DP'] = map_std
# df_test.loc[363:368]
df_train.loc[446:451]

Add a calculation of Coefficient of Variation (variance) to **df_train** test set

In [ ]:
import numpy as np
cv = lambda data: np.std(data, ddof=1) / np.mean(data, axis=0) * 100 
var_group = df_train.groupby('Data_No').apply(cv)
var_group.index.name = None
var_group['Data_No'] = var_group.index
map_var = df_train['Data_No'].map(var_group.set_index('Data_No')['Differential_pressure'])
df_train['cv_DP'] = map_var
# df_test.loc[363:368]
df_train.loc[446:451]

The coefficient of variation is an indication of how far the standard deviation is away from the mean. As we can see it does not add value to our understanding of the data, primarily due the the skewed nature of the `differential_pressure` continuous variable. 
* This re-enforces the understanding that descriptive statistics using the mean may not be preferred measure of central tendency.

**Remove Coefficient of Variation and Add Median to df_train**
* Median is the preferred measure of central tendency to observe in a skewed dataset such as this as it is not as affected by larger values.

In [ ]:
del df_train['cv_DP']
median_group = df_train.groupby('Data_No').median()
median_group.index.name = None
median_group['Data_No'] = median_group.index
map_median = df_train['Data_No'].map(median_group.set_index('Data_No')['Differential_pressure'])
df_train['median_DP'] = map_median
df_train.loc[446:452]

---

#### Review the dataframe with just **A3 Dust** in it, ordered by `filter_balance` as a measure of time to filter failure

Map the size of each bin

In [ ]:
bin_sum = df_train.groupby('Data_No')['Data_No'].count().reset_index(name='bin_Tot')
map_bin = df_train['Data_No'].map(bin_sum.set_index('Data_No')['bin_Tot'])
df_train['bin_Size'] = map_bin
# df_train.loc[38817:38827]

Include a cumulative sum of each **bin size**. This will help us decide on the data bin that reaches **9400** or more total values.

In [ ]:
dust_A3 = df_train[df_train['Dust'] == 1.025]
filter_A3 = dust_A3[dust_A3.Data_No != dust_A3.Data_No.shift(-1)]
df_train_A3 = filter_A3.sort_values(by='Filter_Balance', ascending=True)
df_train_A3['c_Sum'] = df_train_A3['bin_Size'].cumsum()
df_train_A3.head(13)

We can see that in the current dataframe containing only A3 Medium Dust observations, that is ordered by those tests with closest to a completed test to failure:
* **The top 12 data bins (seen at bin 21) would extract a A3 Medium dust training dataset with 9,764 observations**
* We will now perform a further PDA to evaluate the suitability of these further

#### Rank by Standard Deviations, ordered by `std_DP`
The standard deviation is used to measure the spread of values in a sample.

In [ ]:
# dust_A3 = df_train[df_train['Dust'] == 1.025]
# filter_A3 = dust_A3[dust_A3.Data_No != dust_A3.Data_No.shift(-1)]
df_train_A3_std = filter_A3.sort_values(by='std_DP', ascending=True)
df_train_A3_std['c_Sum'] = df_train_A3_std['bin_Size'].cumsum()
df_train_A3_std.head(15)

#### Rank by central tenancy of the Median value, ordered by `median_DP`
* the value of the number in the middle of the dataset

In [ ]:
# dust_A3 = df_train[df_train['Dust'] == 1.025]
# filter_A3 = dust_A3[dust_A3.Data_No != dust_A3.Data_No.shift(-1)]
df_train_A3_median = filter_A3.sort_values(by='median_DP', ascending=True)
df_train_A3_median['c_Sum'] = df_train_A3_median['bin_Size'].cumsum()
df_train_A3_median.head(14)

#### Considerations
* T...

---

### Extract these bins from the df_train dataset

Make a separate frame indicating the bin numbers we wish to extract

In [ ]:
bin_no = df_train_A3['Data_No'].head(12)
bin_no.to_frame()

Use these references to create a dataframe `df_train_cleaned_A3` that is ready for inclusion in our final dataframe `df_train_clean`.
* Note we disregard the cumulative sum measure as it doesn't add value to further calculations

In [ ]:
df_train_copy = df_train
df_train_cleaned_A3 = df_train_copy[df_train_copy['Data_No'].isin(bin_no)]
df_train_cleaned_A3

#### A Quick Review: 
* The Shape we started with

In [ ]:
dust_A3.shape

Shape we have now

In [ ]:
df_train_cleaned_A3.shape

In [ ]:
%matplotlib inline

category_totals = df_train.groupby('Dust')['Differential_pressure'].count().sort_values()
category_totals.plot(kind="barh", title='Proportion of Dust Classes in df_train\n', xlabel='\nObservations', ylabel='Dust Class')
category_totals

### Repeat the cleaning with the next largest set of bins: **A2 Fine Dust**

---

In [ ]:
df_train_A3.describe()

In [ ]:
df_train_A3.median()

Go through each data bin in this dust class and calculate the .median() values

In [ ]:
# df_train.std()
df_train.median()

In [ ]:
df_bin = df_train[df_train['Data_No'] == '12']
# df_bin.median().round(decimals=2)
df_bin

How far the measure is from the median?

In [ ]:
df_train.describe()

### Extract each class and compare distributions

---

#### **Test** dataset

In [ ]:
last_row_test = df_test[df_test.Data_No != df_test.Data_No.shift(-1)]
last_row_descending = last_row_test.sort_values(by='Differential_pressure', ascending=False)
last_row_descending.head(n=10)

Check dataframe distributions

In [ ]:
for df in (df_train, df_test):
    df_numpy = df
    df_numpy.to_numpy()
    name_dataframe(df_numpy)
    print(df_numpy.shape)

---

# Correlation and Power Predictive Score Analysis

---

## Save Datasets

Save the files to /cleaned folder

In [ ]:
import os
try:
  os.makedirs(name='outputs/datasets/cleaned')
except Exception as e:
  print(e)

df_train.to_csv(f'outputs/datasets/cleaned/dfCleanTrain.csv',index=False)
df_test.to_csv(f'outputs/datasets/cleaned/dfCleanTest.csv',index=False)

---

# Conclusions and Next steps

#### Conclusions: 
* 

#### Next Steps:
* Correlation Study
* Feature Engineering

---